In this notebook, we will prepare the data for training with TFOD API. Here's the list of things we are gonna do:
- Calculate `xmax` and `ymax` coordinates and append them to the DataFrame. 
- Rename the `image_id` column to `filename` & add full paths. 
- Add the images that do not have wheat heads to train the model better. 
- Create a class column and fill it with the values of `wheat_head` and `no_wheat_head` accordingly. 
- Create train and validation splits and separate them in DataFrames. 
- Drop the unnecessary columns from the train and validation DataFrames and serialize the new DataFrames in fresh `.csv` files. 
- Generate TFRecords (ideally should be from Terminal) with a stock `generate_tfrecord.py` script. 
- Prepare the label maps. 

In [1]:
from imutils import paths
import pandas as pd
import numpy as np 
import os

In [2]:
train_df = pd.read_csv("train_df.csv")
train_df.head()

,image_id,width,height,bbox,source,xmin,ymin
0,b6ab77fd7,56.0,36.0,"[834.0, 222.0, 56.0, 36.0]",usask_1,834.0,222.0
1,b6ab77fd7,130.0,58.0,"[226.0, 548.0, 130.0, 58.0]",usask_1,226.0,548.0
2,b6ab77fd7,74.0,160.0,"[377.0, 504.0, 74.0, 160.0]",usask_1,377.0,504.0
3,b6ab77fd7,109.0,107.0,"[834.0, 95.0, 109.0, 107.0]",usask_1,834.0,95.0
4,b6ab77fd7,124.0,117.0,"[26.0, 144.0, 124.0, 117.0]",usask_1,26.0,144.0


In [3]:
# Calculating xmax and ymax
train_df["xmax"] = train_df["xmin"] + train_df["width"]
train_df["ymax"] = train_df["ymin"] + train_df["height"]
train_df.head()

,image_id,width,height,bbox,source,xmin,ymin,xmax,ymax
0,b6ab77fd7,56.0,36.0,"[834.0, 222.0, 56.0, 36.0]",usask_1,834.0,222.0,890.0,258.0
1,b6ab77fd7,130.0,58.0,"[226.0, 548.0, 130.0, 58.0]",usask_1,226.0,548.0,356.0,606.0
2,b6ab77fd7,74.0,160.0,"[377.0, 504.0, 74.0, 160.0]",usask_1,377.0,504.0,451.0,664.0
3,b6ab77fd7,109.0,107.0,"[834.0, 95.0, 109.0, 107.0]",usask_1,834.0,95.0,943.0,202.0
4,b6ab77fd7,124.0,117.0,"[26.0, 144.0, 124.0, 117.0]",usask_1,26.0,144.0,150.0,261.0


In [4]:
# Rename the image_id column to filename & add full paths
train_df.rename(columns={
        "image_id":"filename"
    }, inplace=True)

images_w_bbox = train_df["filename"]
images_w_bbox = ["train/" + image_id + ".jpg" for image_id in images_w_bbox]
train_df["filename"] = images_w_bbox
train_df.head()

,filename,width,height,bbox,source,xmin,ymin,xmax,ymax
0,train/b6ab77fd7.jpg,56.0,36.0,"[834.0, 222.0, 56.0, 36.0]",usask_1,834.0,222.0,890.0,258.0
1,train/b6ab77fd7.jpg,130.0,58.0,"[226.0, 548.0, 130.0, 58.0]",usask_1,226.0,548.0,356.0,606.0
2,train/b6ab77fd7.jpg,74.0,160.0,"[377.0, 504.0, 74.0, 160.0]",usask_1,377.0,504.0,451.0,664.0
3,train/b6ab77fd7.jpg,109.0,107.0,"[834.0, 95.0, 109.0, 107.0]",usask_1,834.0,95.0,943.0,202.0
4,train/b6ab77fd7.jpg,124.0,117.0,"[26.0, 144.0, 124.0, 117.0]",usask_1,26.0,144.0,150.0,261.0


In [6]:
# Drop the unnecessary columns, we will return to this step in a moment
train_df.drop(["source", "bbox"], axis=1, inplace=True)
train_df.head()

,filename,width,height,xmin,ymin,xmax,ymax
0,train/b6ab77fd7.jpg,56.0,36.0,834.0,222.0,890.0,258.0
1,train/b6ab77fd7.jpg,130.0,58.0,226.0,548.0,356.0,606.0
2,train/b6ab77fd7.jpg,74.0,160.0,377.0,504.0,451.0,664.0
3,train/b6ab77fd7.jpg,109.0,107.0,834.0,95.0,943.0,202.0
4,train/b6ab77fd7.jpg,124.0,117.0,26.0,144.0,150.0,261.0


In [13]:
# Add a class column 
train_df["class"] = "wheat_head"
train_df.head()

,filename,width,height,xmin,ymin,xmax,ymax,class
0,train/b6ab77fd7.jpg,56.0,36.0,834.0,222.0,890.0,258.0,wheat_head
1,train/b6ab77fd7.jpg,130.0,58.0,226.0,548.0,356.0,606.0,wheat_head
2,train/b6ab77fd7.jpg,74.0,160.0,377.0,504.0,451.0,664.0,wheat_head
3,train/b6ab77fd7.jpg,109.0,107.0,834.0,95.0,943.0,202.0,wheat_head
4,train/b6ab77fd7.jpg,124.0,117.0,26.0,144.0,150.0,261.0,wheat_head


In [5]:
# Images without bounding box
images_w_bbox = train_df["filename"].unique()

all_images = list(paths.list_images("train"))

images_wo_bbox = list(set(all_images) - set(images_w_bbox))
images_wo_bbox[:5]

['train/1a9792bfc.jpg',
 'train/a3ce975cd.jpg',
 'train/69352f3fa.jpg',
 'train/526d737d1.jpg',
 'train/0cf7ef43d.jpg']

In [19]:
# Prepare a separate DataFrame with the images having no bounding boxes
no_wheat_df = pd.DataFrame()
no_wheat_df["filename"] = images_wo_bbox
no_wheat_df["width"] = 0.0
no_wheat_df["height"] = 0.0
no_wheat_df["xmin"] = 0.0
no_wheat_df["ymin"] = 0.0
no_wheat_df["xmax"] = 0.0
no_wheat_df["ymax"] = 0.0
no_wheat_df["class"] = "no_wheat_head"
no_wheat_df.head()

,filename,width,height,xmin,ymin,xmax,ymax,class
0,train/1a9792bfc.jpg,0.0,0.0,0.0,0.0,0.0,0.0,no_wheat_head
1,train/a3ce975cd.jpg,0.0,0.0,0.0,0.0,0.0,0.0,no_wheat_head
2,train/69352f3fa.jpg,0.0,0.0,0.0,0.0,0.0,0.0,no_wheat_head
3,train/526d737d1.jpg,0.0,0.0,0.0,0.0,0.0,0.0,no_wheat_head
4,train/0cf7ef43d.jpg,0.0,0.0,0.0,0.0,0.0,0.0,no_wheat_head


In [20]:
# Concat two DataFrame
total_df = pd.concat([train_df, no_wheat_df], ignore_index=True)
total_df.sample(10)

,filename,width,height,xmin,ymin,xmax,ymax,class
114970,train/f21c0d164.jpg,83.0,115.0,939.0,64.0,1022.0,179.0,wheat_head
64622,train/ba35357cb.jpg,81.0,18.0,476.0,973.0,557.0,991.0,wheat_head
61046,train/866a328af.jpg,120.0,42.0,574.0,743.0,694.0,785.0,wheat_head
13709,train/c00c660fc.jpg,105.0,54.0,230.0,911.0,335.0,965.0,wheat_head
113966,train/9ef3c0a46.jpg,72.0,61.0,47.0,255.0,119.0,316.0,wheat_head
117637,train/832cf59eb.jpg,72.0,64.0,509.0,821.0,581.0,885.0,wheat_head
133168,train/63467d323.jpg,71.0,84.0,890.0,532.0,961.0,616.0,wheat_head
13587,train/2ed94451a.jpg,93.0,121.0,382.0,58.0,475.0,179.0,wheat_head
31940,train/a750327c3.jpg,74.0,25.0,950.0,0.0,1024.0,25.0,wheat_head
110049,train/ecd58cc0a.jpg,59.0,108.0,737.0,145.0,796.0,253.0,wheat_head


In [23]:
# What is the new distribution now?
total_df["class"].value_counts()

wheat_head       147793
no_wheat_head        49
Name: class, dtype: int64

In [26]:
total_df[total_df["class"]=="no_wheat_head"].head()

,filename,width,height,xmin,ymin,xmax,ymax,class
147793,train/1a9792bfc.jpg,0.0,0.0,0.0,0.0,0.0,0.0,no_wheat_head
147794,train/a3ce975cd.jpg,0.0,0.0,0.0,0.0,0.0,0.0,no_wheat_head
147795,train/69352f3fa.jpg,0.0,0.0,0.0,0.0,0.0,0.0,no_wheat_head
147796,train/526d737d1.jpg,0.0,0.0,0.0,0.0,0.0,0.0,no_wheat_head
147797,train/0cf7ef43d.jpg,0.0,0.0,0.0,0.0,0.0,0.0,no_wheat_head


In [34]:
# Prepare the splits
from sklearn.model_selection import train_test_split

train, valid = train_test_split(total_df, test_size=0.1, 
                                stratify=total_df['class'], random_state=666)
print("Training samples:", train.shape[0])
print("Validation samples:", valid.shape[0])

Training samples: 133057
Validation samples: 14785


In [35]:
train = train.reset_index(drop=True)
train.head()

,filename,width,height,xmin,ymin,xmax,ymax,class
0,train/fd30a89bb.jpg,82.0,105.0,747.0,46.0,829.0,151.0,wheat_head
1,train/ea3537563.jpg,29.0,90.0,989.0,501.0,1018.0,591.0,wheat_head
2,train/fd30a89bb.jpg,102.0,89.0,847.0,374.0,949.0,463.0,wheat_head
3,train/5530bd2c3.jpg,91.0,102.0,193.0,164.0,284.0,266.0,wheat_head
4,train/fe8876602.jpg,47.0,63.0,870.0,961.0,917.0,1024.0,wheat_head


In [36]:
valid = valid.reset_index(drop=True)
valid.head()

,filename,width,height,xmin,ymin,xmax,ymax,class
0,train/4ffeedce2.jpg,65.0,58.0,959.0,949.0,1024.0,1007.0,wheat_head
1,train/8c8ab21bc.jpg,58.0,63.0,329.0,767.0,387.0,830.0,wheat_head
2,train/42f5fe6a5.jpg,75.0,146.0,616.0,361.0,691.0,507.0,wheat_head
3,train/8c627af48.jpg,54.0,49.0,733.0,511.0,787.0,560.0,wheat_head
4,train/967ff723d.jpg,82.0,35.0,808.0,950.0,890.0,985.0,wheat_head


In [37]:
# Serialize the dataframes
train.to_csv("new_train_df.csv")
valid.to_csv("valid_df.csv")

Now, we will be utilizing the `generate_tfrecords.py` script from the [TFOD API GitHub repository](https://github.com/tensorflow/models/tree/master/research/object_detection) to convert the `.csv` records to TFRecords that are compatible with the TFOD API. A [step-by-step guide](https://www.kaggle.com/aakashnain/eda2modelling-tf-object-detection-api) is available here. 

After you create the TFRecords, verify their sizes. Mine looks like so - 
```
$ ls -lh *.record
-rw-r--r-- 1 jupyter jupyter 609M May  6 05:45 train.record
-rw-r--r-- 1 jupyter jupyter 574M May  6 05:46 valid.record
```

In [47]:
# Preparing the label maps
LABEL_ENCODINGS = {
    "no_wheat_head": 2,
    "wheat_head": 1
}

f = open("label_map.pbtxt", "w")

for (k, v) in LABEL_ENCODINGS.items():
    # construct the class information and write to file
    item = ("item {\n"
            "\tid: " + str(v) + "\n"
            "\tname: '" + k + "'\n"
            "}\n")
    f.write(item)

# close the output classes file
f.close()

In [46]:
!cat label_map.pbtxt

item {
	id: 0
	name: 'no_wheat_head'
}
item {
	id: 1
	name: 'wheat_head'
}
